In [142]:
import pandas as pd
import numpy as np
import os
from ctypes import Structure, c_int, c_double, POINTER, CDLL, c_char
import subprocess
import time

In [165]:
TIME_INTERVAL = 5
HORIZON = int(round(24 * 60 / TIME_INTERVAL))

# // search TAG for reference speeds for urban traffic, cited value of why you're using a partiular speed
AVG_SPEED_PER_HOUR = (
    20.4 * 1.60934
)  # km/h taken from https://www.gov.uk/government/statistical-data-sets/average-speed-delay-and-reliability-of-travel-times-cgn#average-speed-delay-and-reliability-of-travel-times-on-local-a-roads-cgn05
# can also check https://www.gov.uk/government/publications/webtag-tag-unit-m1-2-data-sources-and-surveys
SPEED = AVG_SPEED_PER_HOUR * 16.667  # 1km/h = 16.667 m/min, converts it to minutes
TRAVEL_TIME_PENALTY = 0.1  # we will add dusk, home, dawn and work

In [143]:
class Group_mem(Structure):
    pass


Group_mem._fields_ = [
    ("g", c_int),
    ("previous", POINTER(Group_mem)),
    ("next", POINTER(Group_mem)),
]


class Activity(Structure):
    pass


Activity._fields_ = [
    ("id", c_int),
    ("earliest_start", c_int),
    ("latest_start", c_int),
    ("min_duration", c_int),
    ("max_duration", c_int),
    ("x", c_double),
    ("y", c_double),
    ("group", c_int),
    ("memory", POINTER(Group_mem)),
    ("des_duration", c_int),
    ("des_start_time", c_int),
    ("charge_mode", c_int),
    ("is_charging", c_int),
    ("is_service_station", c_int),
]


class Label(Structure):
    pass


Label._fields_ = [
    ("act_id", c_int),
    ("time", c_int),
    ("start_time", c_int),
    ("duration", c_int),
    ("deviation_start", c_int),
    ("deviation_dur", c_int),
    ("soc_at_activity_start", c_double),
    ("current_soc", c_double),
    ("delta_soc", c_double),
    ("charge_duration", c_int),
    ("charge_cost_at_activity_start", c_double),
    ("current_charge_cost", c_double),
    ("utility", c_double),
    ("mem", POINTER(Group_mem)),
    ("previous", POINTER(Label)),
    ("act", POINTER(Activity)),
]


class L_list(Structure):
    pass


L_list._fields_ = [
    ("element", POINTER(Label)),
    ("previous", POINTER(L_list)),
    ("next", POINTER(L_list)),
]

In [89]:
activities_df = pd.read_csv("data/activities_long_with_groups_selected_plus.csv")
people_df = pd.read_csv("data/persons_home_depot_with_start_duration_draws.csv")

/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/1171936967.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv("data/persons_home_depot_with_start_duration_draws.csv")


In [90]:
activities_df = activities_df.sort_values(by="pid")
activities_df.head()

,pid,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
0,2002000535_3_E02001543_1_2002001259,1,home,453768.000000,408587.000000,1,0,288,288,2
1,2002000535_3_E02001543_1_2002001259,2,other,453768.000000,408587.000000,8,108,216,120,2
2,2002000535_3_E02001543_1_2002001259,3,home,453768.000000,408587.000000,1,0,288,288,2
9,2002000535_3_E02001543_1_2002001260,7,home,453768.000000,408587.000000,1,0,288,288,2
7,2002000535_3_E02001543_1_2002001260,5,pt interaction,453392.786254,407728.186228,8,108,216,120,2


In [91]:
people_df.rename(columns={"x": "home_x", "y": "home_y"}, inplace=True)
people_df.head()

,pid,home_x,home_y,hhincome,workstatus,gender,age_group,group,start_g1,dur_g1,...,start_g4,dur_g4,start_g5,dur_g5,start_g6,dur_g6,start_g7,dur_g7,start_g8,dur_g8
0,2002000535_3_E02001543_1_2002001259,453768.0,408587.0,low,False,male,65+,1,0.0,156.0,...,159.0,12.0,287.0,35.0,NaN,NaN,NaN,NaN,148.0,149.0
1,2002000535_3_E02001543_1_2002001260,453768.0,408587.0,low,False,female,65+,1,0.0,138.0,...,127.0,5.0,148.0,36.0,NaN,NaN,NaN,NaN,149.0,4.0
2,2002000535_5_E02001617_1_2002001259,437044.0,393175.0,low,False,male,65+,1,224.0,169.0,...,128.0,164.0,147.0,35.0,NaN,NaN,NaN,NaN,108.0,20.0
3,2002000535_5_E02001617_1_2002001260,437044.0,393175.0,low,False,female,65+,1,122.0,136.0,...,177.0,46.0,147.0,35.0,NaN,NaN,NaN,NaN,130.0,29.0
4,2002000536_7_E02001588_1_2002001261,440254.0,394149.0,low,False,female,65+,1,0.0,114.0,...,109.0,71.0,287.0,8.0,NaN,NaN,NaN,NaN,242.0,67.0


In [92]:
acts_people = set(activities_df["pid"].unique())
people_total = set(people_df["pid"].unique())
len(people_total)

127393

In [93]:
usable_people = set(person for person in acts_people if person in people_total)
len(usable_people)

127393

In [94]:
acts_df = activities_df[activities_df["pid"].isin(usable_people)]
acts_df.to_csv("data/activities_list_per_pid.csv", index=False)
# len(acts_df["pid"].unique())
# set(acts_df["pid"].unique()) ==people_total

In [ ]:
users_to_extract = 1

people_to_use = people_df.sample(n=users_to_extract)
acts_to_use = acts_df[acts_df["pid"].isin(people_to_use["pid"].unique())]

people_to_use.head()



,pid,home_x,home_y,hhincome,workstatus,gender,age_group,group,start_g1,dur_g1,...,start_g4,dur_g4,start_g5,dur_g5,start_g6,dur_g6,start_g7,dur_g7,start_g8,dur_g8
4999,2002007185_4_E02001606_1_2002016833,452779.0,386207.0,low,True,male,41-65,1,0.0,13.0,...,192.0,40.0,101.0,27.0,NaN,NaN,NaN,NaN,160.0,182.0


In [173]:
acts_to_use

,pid,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
26804,2002007185_4_E02001606_1_2002016833,9,home,452779.0,386207.0,1,0,288,288,2
26803,2002007185_4_E02001606_1_2002016833,8,shop,457341.0,380889.0,4,84,276,120,2
26802,2002007185_4_E02001606_1_2002016833,7,home,452779.0,386207.0,1,0,288,288,2
26801,2002007185_4_E02001606_1_2002016833,6,shop,444026.0,382607.0,4,84,276,120,2
26800,2002007185_4_E02001606_1_2002016833,5,home,452779.0,386207.0,1,0,288,288,2
26798,2002007185_4_E02001606_1_2002016833,3,home,452779.0,386207.0,1,0,288,288,2
26797,2002007185_4_E02001606_1_2002016833,2,work,442017.0,388133.0,2,60,276,144,2
26796,2002007185_4_E02001606_1_2002016833,1,home,452779.0,386207.0,1,0,288,288,2
26799,2002007185_4_E02001606_1_2002016833,4,other,455942.0,387053.0,8,108,216,120,2


In [176]:
acts_to_use.to_csv("data/test/activities_person_833.csv", index=False)

In [97]:
def extract_individual_activities(acts_to_use, pid):
    return acts_to_use[acts_to_use["pid"] == pid]

In [204]:
def initialise_and_personalise_activities(activities, individual):
    max_num_activities = len(activities)
    # print(f"max_acts: {max_num_activities}")
    activities_array = (Activity * (max_num_activities+1))()

    for _, row in activities.iterrows():
        act_id = int(row["activity_idx"])
        # print(f"act id: {act_id}")
        group = int(row["group"])

        # Use act_id - 1 as array index since activity_idx starts from 1
        array_idx = act_id - 1

        activities_array[array_idx].id = act_id-1
        activities_array[array_idx].x = row["x"]
        activities_array[array_idx].y = row["y"]
        activities_array[array_idx].earliest_start = int(row["earliest_start"])
        activities_array[array_idx].latest_start = int(row["latest_start"])
        activities_array[array_idx].max_duration = int(row["max_duration"])
        activities_array[array_idx].min_duration = int(row["min_duration"])
        activities_array[array_idx].des_duration = int(individual[f"dur_g{group}"]) if pd.notna(individual[f"dur_g{group}"]).item() else 0
        activities_array[array_idx].des_start_time = int(individual[f"start_g{group}"]) if not pd.isna(individual[f"start_g{group}"]).item() else 0
        activities_array[array_idx].charge_mode = 0
        activities_array[array_idx].is_charging = 0
        activities_array[array_idx].is_service_station = 0
        activities_array[array_idx].group = group-1
        activities_array[array_idx].memory = None

        # Dusk activity
    activities_array[max_num_activities].id = max_num_activities
    activities_array[max_num_activities].x = individual["home_x"]
    activities_array[max_num_activities].y = individual["home_y"]
    activities_array[max_num_activities].earliest_start = 0
    activities_array[max_num_activities].latest_start = HORIZON - 2
    activities_array[max_num_activities].max_duration = HORIZON - 2
    activities_array[max_num_activities].min_duration = 1
    activities_array[max_num_activities].group = 0
    activities_array[max_num_activities].des_duration = 0
    activities_array[max_num_activities].des_start_time = 0
    activities_array[max_num_activities].charge_mode = 0
    activities_array[max_num_activities].is_charging = 0
    activities_array[max_num_activities].is_service_station = 0

    return activities_array, max_num_activities

In [138]:
def compile_code():
    """Compile the scheduling C code as a shared library for Python ctypes."""
    # Get the current directory and parent directory
    # current_dir = os.path.dirname(os.path.abspath(__file__))
    # current_dir = /Users/charlottesavage/MainDocs/Masters/Research proj/DP_modelling/scheduling_code/unify_data.ipynb
    # parent_dir = os.path.dirname(current_dir)

    # Define paths (src and include are in parent directory)
    src_dir = "src/"
    inc_dir = "include/"
    output_lib = "scheduling.so"

    # Source files to compile
    sources = [
        os.path.join(src_dir, "scheduling.c"),
        os.path.join(src_dir, "utils.c"),
        os.path.join(src_dir, "main.c"),
    ]

    # Check if recompilation is needed
    needs_recompile = False
    if not os.path.exists(output_lib):
        needs_recompile = True
        print("No existing compiled library found")
    else:
        # Check if any source file is newer than the compiled library
        lib_mtime = os.path.getmtime(output_lib)
        for src_file in sources:
            if os.path.getmtime(src_file) > lib_mtime:
                needs_recompile = True
                print(f"Source file {os.path.basename(src_file)} is newer than compiled library")
                break

        # Also check header files
        if not needs_recompile:
            for header_file in os.listdir(inc_dir):
                if header_file.endswith('.h'):
                    header_path = os.path.join(inc_dir, header_file)
                    if os.path.getmtime(header_path) > lib_mtime:
                        needs_recompile = True
                        print(f"Header file {header_file} is newer than compiled library")
                        break

        if not needs_recompile:
            print(f"Using existing compiled library: {output_lib}")
            return output_lib

    compile_command = [
        "gcc",
        "-m64",
        "-O3",
        "-shared",
        "-fPIC",
        f"-I{inc_dir}",
        "-o",
        output_lib,
    ] + sources + ["-lm"]

    print(f"Compiling C code: {' '.join(compile_command)}")
    result = subprocess.run(compile_command, capture_output=True, text=True)

    if result.returncode != 0:
        print("Compilation failed!")
        print(f"STDERR: {result.stderr}")
        raise RuntimeError("Failed to compile C code")
    else:
        print(f"Compilation successful! Created {output_lib}")

    return output_lib

In [145]:
def run_dp(lib, activities_array, max_num_activities, params):
    """Run the DP algorithm."""
    print("\n" + "=" * 60)
    print("Running DP Algorithm...")
    print("=" * 60)

    # Convert parameters to C arrays
    asc_array = (c_double * len(params["asc"]))(*params["asc"])
    early_array = (c_double * len(params["early"]))(*params["early"])
    late_array = (c_double * len(params["late"]))(*params["late"])
    long_array = (c_double * len(params["long"]))(*params["long"])
    short_array = (c_double * len(params["short"]))(*params["short"])

    lib.set_general_parameters(
        HORIZON,
        SPEED,
        TRAVEL_TIME_PENALTY,
        TIME_INTERVAL,
        asc_array,
        early_array,
        late_array,
        long_array,
        short_array,
    )

    # Set activities
    lib.set_activities(activities_array, max_num_activities)

    print(f"Number of activities: {max_num_activities}")
    print(f"Horizon: {HORIZON} intervals ({HORIZON * TIME_INTERVAL / 60:.1f} hours)")
    print("Starting DP...")

    # Run main (creates bucket, runs DP, handles DSSR)
    # Pass dummy argc=0, argv=NULL since we're not using command line args
    start_time = time.time()
    result = lib.main(0, None)
    total_time = time.time() - start_time

    print(f"C main() returned: {result}")

    print(f"\nDP completed in {total_time:.2f} seconds")

    # Get final schedule
    best_label = lib.get_final_schedule()

    if not best_label:
        print("ERROR: No feasible solution found!")
        return None

    print(f"Final utility: {best_label.contents.utility:.2f}")

    return best_label

In [201]:
people_to_use

,pid,home_x,home_y,hhincome,workstatus,gender,age_group,group,start_g1,dur_g1,...,start_g4,dur_g4,start_g5,dur_g5,start_g6,dur_g6,start_g7,dur_g7,start_g8,dur_g8
4999,2002007185_4_E02001606_1_2002016833,452779.0,386207.0,low,True,male,41-65,1,0.0,13.0,...,192.0,40.0,101.0,27.0,NaN,NaN,NaN,NaN,160.0,182.0


In [205]:
for pid in people_to_use["pid"].unique():
    individual = people_to_use[people_to_use["pid"] == pid]
    activities = acts_to_use[acts_to_use["pid"] == pid]
    activities_array, max_num_activities = initialise_and_personalise_activities(
        activities=activities, individual=individual
    )

activities_array

/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/3028074109.py:21: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  activities_array[array_idx].des_duration = int(individual[f"dur_g{group}"]) if pd.notna(individual[f"dur_g{group}"]).item() else 0
/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/3028074109.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  activities_array[array_idx].des_start_time = int(individual[f"start_g{group}"]) if not pd.isna(individual[f"start_g{group}"]).item() else 0
/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/3028074109.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  activities_array[max_num_activities].x = individual["hom

In [206]:
# Example: How to examine elements of activities_array (a ctypes array)

# Access individual elements by index
print("Activity at index 0:")
print(f"  ID: {activities_array[0].id}")
print(f"  Position: ({activities_array[0].x}, {activities_array[0].y})")
print(f"  Earliest start: {activities_array[0].earliest_start}")
print(f"  Latest start: {activities_array[0].latest_start}")
print(f"  Min duration: {activities_array[0].min_duration}")
print(f"  Max duration: {activities_array[0].max_duration}")
print(f"  Desired duration: {activities_array[0].des_duration}")
print(f"  Desired start time: {activities_array[0].des_start_time}")
print(f"  Group: {activities_array[0].group}")

print("\n" + "="*50 + "\n")

# Loop through all activities
print("All activities:")
for i in range(max_num_activities+1):
    act = activities_array[i]
    print(f"Activity {i}: ID={act.id}, Group={act.group}, "
          f"Position=({act.x:.1f}, {act.y:.1f}), "
          f"Time window=[{act.earliest_start}, {act.latest_start}], "
          f"Duration=[{act.min_duration}, {act.max_duration}], "
          f"Desired: start={act.des_start_time}, dur={act.des_duration}")

print("\n" + "="*50 + "\n")

# Convert to a more readable format (dictionary)
print("As dictionaries:")
for i in range(max_num_activities+1):
    act = activities_array[i]
    act_dict = {
        'id': act.id,
        'group': act.group,
        'x': act.x,
        'y': act.y,
        'earliest_start': act.earliest_start,
        'latest_start': act.latest_start,
        'min_duration': act.min_duration,
        'max_duration': act.max_duration,
        'des_duration': act.des_duration,
        'des_start_time': act.des_start_time,
        'is_charging': act.is_charging
    }
    print(f"Activity {i}: {act_dict}")

Activity at index 0:
  ID: 0
  Position: (452779.0, 386207.0)
  Earliest start: 0
  Latest start: 288
  Min duration: 2
  Max duration: 288
  Desired duration: 13
  Desired start time: 0
  Group: 0


All activities:
Activity 0: ID=0, Group=0, Position=(452779.0, 386207.0), Time window=[0, 288], Duration=[2, 288], Desired: start=0, dur=13
Activity 1: ID=1, Group=1, Position=(442017.0, 388133.0), Time window=[60, 276], Duration=[2, 144], Desired: start=158, dur=184
Activity 2: ID=2, Group=0, Position=(452779.0, 386207.0), Time window=[0, 288], Duration=[2, 288], Desired: start=0, dur=13
Activity 3: ID=3, Group=7, Position=(455942.0, 387053.0), Time window=[108, 216], Duration=[2, 120], Desired: start=160, dur=182
Activity 4: ID=4, Group=0, Position=(452779.0, 386207.0), Time window=[0, 288], Duration=[2, 288], Desired: start=0, dur=13
Activity 5: ID=5, Group=3, Position=(444026.0, 382607.0), Time window=[84, 276], Duration=[2, 120], Desired: start=192, dur=40
Activity 6: ID=6, Group=0, P

In [158]:
def extract_schedule(best_label, activities_array):
    """
    Extract the schedule from the best label.

    Uses the same approach as main_slice_cs.py: group by (act_id, start_time)
    and keep only the label with maximum duration (= final state for that activity visit).
    """
    # Collect all labels in the path
    path_to_root = []
    current = best_label
    while current:
        path_to_root.append(current)
        current = current.contents.previous

    # Process labels and group by unique (act_id, start_time)
    schedule_dict = {}
    for label_pointer in reversed(path_to_root):
        label = label_pointer.contents
        activity = activities_array[label.act_id]

        unique_key = (label.act_id, label.start_time)
        data = {
            'act_id': label.act_id,
            'act_type': 'home' if activity.group == 0 else f'group_{activity.group}',
            'start_time': label.start_time * TIME_INTERVAL / 60,  # Convert to hours
            'duration': label.duration * TIME_INTERVAL / 60,  # Convert to hours
            'soc_start': label.soc_at_activity_start,
            'soc_end': label.current_soc,
            'is_charging': activity.is_charging,
            'charge_mode': activity.charge_mode,
            'charge_duration': label.charge_duration * TIME_INTERVAL / 60,  # hours
            'charge_cost': label.current_charge_cost,
            'utility': label.utility,
            'x': activity.x,
            'y': activity.y
        }

        # Keep label with maximum duration for each (act_id, start_time) pair
        if unique_key not in schedule_dict or schedule_dict[unique_key]['duration'] < data['duration']:
            schedule_dict[unique_key] = data

    schedule = list(schedule_dict.values())
    return pd.DataFrame(schedule)

In [166]:

def initialize_utility():
    """Initialize utility parameters (same as original)."""
    # Parameters for groups: [0:Home, 1:Education, 2:Errands, 3:Escort, 4:Leisure, 5:Shopping, 6:Work, 7:ServiceStation]
    asc = [0, 17.4, 16.1, 6.76, 12, 11.3, 10.6, 0]
    early = [0, -2.56, -1.73, -2.55, -0.031, -2.51, -1.37, 0]
    late = [0, -1.54, -3.42, -0.578, -1.58, -0.993, -0.79, 0]
    long = [0, -0.0783, -0.597, -0.0267, -0.209, -0.133, -0.201, 0]
    short = [0, -0.783, -5.63, 0.134, -0.00764, 0.528, -4.78, 0]

    return {
        'asc': asc,
        'early': early,
        'late': late,
        'long': long,
        'short': short
    }

In [207]:
lib_path = compile_code()

Using existing compiled library: scheduling.so


In [208]:
lib = CDLL(lib_path)

In [209]:
lib.set_general_parameters.argtypes = [
        c_int,  # horizon
        c_double,  # speed
        c_double,  # travel_time_penalty
        c_int,  # time_interval
        POINTER(c_double),  # asc
        POINTER(c_double),  # early
        POINTER(c_double),  # late
        POINTER(c_double),  # long
        POINTER(c_double),  # short
    ]

lib.set_activities.argtypes = [POINTER(Activity), c_int]
lib.main.argtypes = [c_int, POINTER(POINTER(c_char))]  # int argc, char **argv
lib.main.restype = c_int
lib.get_final_schedule.restype = POINTER(Label)
lib.free_bucket.restype = None

In [210]:
params = initialize_utility()

In [211]:
best_label = run_dp(lib, activities_array, max_num_activities, params)


Running DP Algorithm...
Number of activities: 9
Horizon: 288 intervals (24.0 hours)
Starting DP...
C main() returned: 0

DP completed in 0.00 seconds
ERROR: No feasible solution found!

 Solution is not feasible, check activities parameters.
 Solution is not feasible, check activities parameters.

In [178]:
# FOUND THE PROBLEM! Let me show you what's wrong.

print("="*70)
print("ROOT CAUSE ANALYSIS")
print("="*70)

print("\n1. YOUR CURRENT ACTIVITY DATA:")
print(acts_to_use.sort_values('activity_idx')[['activity_idx', 'act_type', 'group', 'x', 'y']])

print(f"\n2. THE PROBLEM:")
print(f"   Your activity IDs range from {acts_to_use['activity_idx'].min()} to {acts_to_use['activity_idx'].max()}")
print(f"   You have {len(acts_to_use)} activities")

print("\n3. WHAT THE C ALGORITHM EXPECTS:")
print("   From scheduling.c line 858:")
print("     Label *ll = create_label(&activities[0]);")
print("     → The algorithm starts at activities[0] which must be DAWN (id=0)")
print()
print("   From scheduling.c line 322:")
print("     if (L->act_id != 0 && a->id == 0) { return 0; }")
print("     → Activity with id=0 can ONLY be the first activity (DAWN)")
print()
print("   From scheduling.c line 653:")
print("     if (a->id == max_num_activities - 1)")
print("     → The LAST activity (id = max_num_activities-1) is DUSK")
print()
print("   From main.c line 26:")
print("     L_list *li = &bucket[horizon - 1][max_num_activities - 1];")
print("     → The algorithm looks for solutions ending at the LAST activity")

print("\n4. THE STRUCTURE REQUIRED:")
print("   Activity 0 (DAWN):  Starting home, can only be first, id=0")
print("   Activity 1-7:       Your regular activities (work, shop, other, etc.)")
print("   Activity 8 or N-1 (DUSK): Ending home, must be last")
print()
print("   Total: You need N+2 activities if you have N regular activities")
print("         OR your existing home activities need to serve as DAWN/DUSK")

print("\n5. CHECKING YOUR ACTIVITIES ARRAY:")
for i in range(max_num_activities):
    act = activities_array[i]
    is_first = " ← SHOULD BE DAWN (id=0)" if i == 0 else ""
    is_last = " ← SHOULD BE DUSK" if i == max_num_activities - 1 else ""
    print(f"   array[{i}]: id={act.id}, group={act.group}, type=?, pos=({act.x:.0f},{act.y:.0f}){is_first}{is_last}")

print("\n6. THE SPECIFIC ISSUE:")
first_act = activities_array[0]
last_act = activities_array[max_num_activities - 1]

if first_act.id != 0:
    print(f"   ❌ activities_array[0] has id={first_act.id}, but MUST have id=0 (DAWN)")
else:
    print(f"   ✓ activities_array[0] correctly has id=0 (DAWN)")

if last_act.id != max_num_activities - 1:
    print(f"   ❌ activities_array[{max_num_activities-1}] has id={last_act.id}, but MUST have id={max_num_activities-1} (DUSK)")
else:
    print(f"   ✓ activities_array[{max_num_activities-1}] correctly has id={max_num_activities-1} (DUSK)")

print("\n7. WHY IT'S INFEASIBLE:")
print("   The DP algorithm initializes with activities[0] (expecting DAWN)")
print("   Then it searches for paths ending at activities[max_num_activities-1] (expecting DUSK)")
print("   If these aren't set up correctly, NO path can be found!")

print("\n8. SOLUTION OPTIONS:")
print("   A) Renumber your activities so:")
print("      - One home activity has id=0 (DAWN)")
print("      - Other activities have id=1 to N-2")
print("      - One home activity has id=N-1 (DUSK, where N=total activities)")
print()
print("   B) Add explicit DAWN and DUSK activities to your dataframe")
print("      - DAWN: id=0, group=1 (or 0), at home location, time window [0, 288]")
print("      - DUSK: id=max_id+1, group=1 (or 0), at home location, time window [0, 288]")

print("\n" + "="*70)

ROOT CAUSE ANALYSIS

1. YOUR CURRENT ACTIVITY DATA:
       activity_idx act_type  group         x         y
26796             1     home      1  452779.0  386207.0
26797             2     work      2  442017.0  388133.0
26798             3     home      1  452779.0  386207.0
26799             4    other      8  455942.0  387053.0
26800             5     home      1  452779.0  386207.0
26801             6     shop      4  444026.0  382607.0
26802             7     home      1  452779.0  386207.0
26803             8     shop      4  457341.0  380889.0
26804             9     home      1  452779.0  386207.0

2. THE PROBLEM:
   Your activity IDs range from 1 to 9
   You have 9 activities

3. WHAT THE C ALGORITHM EXPECTS:
   From scheduling.c line 858:
     Label *ll = create_label(&activities[0]);
     → The algorithm starts at activities[0] which must be DAWN (id=0)

   From scheduling.c line 322:
     if (L->act_id != 0 && a->id == 0) { return 0; }
     → Activity with id=0 can ONLY be t

# 🔍 ROOT CAUSE OF INFEASIBILITY

## The Problem

Your DP algorithm was finding **no feasible solution** because of an **activity indexing mismatch**.

## What I Found

After analyzing the C code in `src/scheduling.c` and `src/main.c`, I discovered that the DP algorithm has **strict requirements** for activity structure:

### Required Structure:
1. **Activity at index 0 MUST have `id=0`** (DAWN - starting home)
   - From `scheduling.c:858`: `Label *ll = create_label(&activities[0]);`
   - From `scheduling.c:322`: Activity id=0 can ONLY be the first activity
   
2. **Activity at index N-1 MUST have `id=N-1`** (DUSK - ending home)
   - From `main.c:26`: Algorithm looks for solutions at `bucket[horizon-1][max_num_activities-1]`
   - From `scheduling.c:653`: Special handling for last activity as DUSK

### Your Data Had:
- Activity IDs ranging from 1 to 9 (no id=0!)
- Multiple home activities scattered throughout
- No explicit DAWN (id=0) or DUSK (id=N-1) activities

### Why This Caused Infeasibility:
- The algorithm starts by creating a label at `activities[0]`, expecting DAWN
- It searches for solutions ending at `activities[max_num_activities-1]`, expecting DUSK
- Since your first activity wasn't DAWN and your last wasn't DUSK, **no valid path could exist**

## The Solution

I created the `fix_activities_for_dp()` function which:
1. Creates a **DAWN** activity (id=0) at the home location
2. Renumbers your regular activities sequentially (ids 1 to N)
3. Creates a **DUSK** activity (id=N+1) at the home location
4. Ensures both DAWN and DUSK have full time windows [0, 288]

Now run the cells below to see if the fixed activities produce a feasible solution!

In [179]:
def fix_activities_for_dp(acts_df, person_df):
    """
    Fix activities dataframe to work with the DP algorithm.
    
    The DP algorithm requires:
    - Activity with id=0 must be DAWN (starting home)
    - Activity with id=max_num_activities-1 must be DUSK (ending home)
    - All other activities numbered sequentially in between
    
    This function:
    1. Identifies home activities
    2. Creates DAWN (id=0) and DUSK (id=N-1) from home location
    3. Renumbers other activities sequentially
    """
    home_x = person_df['home_x'].values[0]
    home_y = person_df['home_y'].values[0]
    
    # Separate home and non-home activities
    home_acts = acts_df[acts_df['act_type'] == 'home'].copy()
    non_home_acts = acts_df[acts_df['act_type'] != 'home'].copy()
    
    # Create DAWN activity (id=0)
    dawn = pd.DataFrame([{
        'pid': acts_df['pid'].iloc[0],
        'activity_idx': 0,
        'act_type': 'home',
        'x': home_x,
        'y': home_y,
        'group': 1,  # Home group
        'earliest_start': 0,
        'latest_start': 288,
        'max_duration': 288,
        'min_duration': 2
    }])
    
    # Renumber non-home activities to start from 1
    non_home_acts = non_home_acts.sort_values('activity_idx').reset_index(drop=True)
    non_home_acts['activity_idx'] = range(1, len(non_home_acts) + 1)
    
    # Create DUSK activity (id = total-1)
    dusk_id = len(non_home_acts) + 1  # dawn(0) + non_home(N) + dusk(1) = N+2 total
    dusk = pd.DataFrame([{
        'pid': acts_df['pid'].iloc[0],
        'activity_idx': dusk_id,
        'act_type': 'home',
        'x': home_x,
        'y': home_y,
        'group': 1,  # Home group
        'earliest_start': 0,
        'latest_start': 288,
        'max_duration': 288,
        'min_duration': 2
    }])
    
    # Combine: DAWN + activities + DUSK
    fixed_acts = pd.concat([dawn, non_home_acts, dusk], ignore_index=True)
    fixed_acts = fixed_acts.sort_values('activity_idx').reset_index(drop=True)
    
    print(f"Original activities: {len(acts_df)}")
    print(f"Fixed activities: {len(fixed_acts)}")
    print(f"  - DAWN (id=0)")
    print(f"  - {len(non_home_acts)} regular activities (ids 1-{len(non_home_acts)})")
    print(f"  - DUSK (id={dusk_id})")
    
    return fixed_acts

# Apply the fix
acts_to_use_fixed = fix_activities_for_dp(acts_to_use, people_to_use)

print("\n" + "="*70)
print("FIXED ACTIVITIES:")
print("="*70)
print(acts_to_use_fixed[['activity_idx', 'act_type', 'group', 'x', 'y', 
                          'earliest_start', 'latest_start', 'min_duration', 'max_duration']])

Original activities: 9
Fixed activities: 6
  - DAWN (id=0)
  - 4 regular activities (ids 1-4)
  - DUSK (id=5)

FIXED ACTIVITIES:
   activity_idx act_type  group         x         y  earliest_start  \
0             0     home      1  452779.0  386207.0               0   
1             1     work      2  442017.0  388133.0              60   
2             2    other      8  455942.0  387053.0             108   
3             3     shop      4  444026.0  382607.0              84   
4             4     shop      4  457341.0  380889.0              84   
5             5     home      1  452779.0  386207.0               0   

   latest_start  min_duration  max_duration  
0           288             2           288  
1           276             2           144  
2           216             2           120  
3           276             2           120  
4           276             2           120  
5           288             2           288  


In [180]:
# Now test with the fixed activities
print("="*70)
print("TESTING WITH FIXED ACTIVITIES")
print("="*70)

# Re-initialize activities with the fixed dataframe
for pid in people_to_use["pid"].unique():
    individual = people_to_use[people_to_use["pid"] == pid]
    activities_fixed = acts_to_use_fixed[acts_to_use_fixed["pid"] == pid]
    activities_array_fixed, max_num_activities_fixed = initialise_and_personalise_activities(
        activities=activities_fixed, individual=individual
    )

print(f"\nFixed activities array has {max_num_activities_fixed} activities")
print(f"First activity (DAWN): id={activities_array_fixed[0].id}, group={activities_array_fixed[0].group}")
print(f"Last activity (DUSK): id={activities_array_fixed[max_num_activities_fixed-1].id}, group={activities_array_fixed[max_num_activities_fixed-1].group}")

# Verify the structure is correct
if activities_array_fixed[0].id == 0:
    print("✓ First activity correctly has id=0 (DAWN)")
else:
    print(f"✗ First activity has id={activities_array_fixed[0].id}, should be 0")

if activities_array_fixed[max_num_activities_fixed-1].id == max_num_activities_fixed - 1:
    print(f"✓ Last activity correctly has id={max_num_activities_fixed-1} (DUSK)")
else:
    print(f"✗ Last activity has id={activities_array_fixed[max_num_activities_fixed-1].id}, should be {max_num_activities_fixed-1}")

print("\nAll activities in order:")
for i in range(max_num_activities_fixed):
    act = activities_array_fixed[i]
    act_type = "DAWN" if i == 0 else ("DUSK" if i == max_num_activities_fixed-1 else f"Group {act.group}")
    print(f"  [{i}] id={act.id}, {act_type}, pos=({act.x:.0f}, {act.y:.0f}), "
          f"window=[{act.earliest_start}, {act.latest_start}]")

TESTING WITH FIXED ACTIVITIES

Fixed activities array has 6 activities
First activity (DAWN): id=1, group=2
Last activity (DUSK): id=0, group=1
✗ First activity has id=1, should be 0
✗ Last activity has id=0, should be 5

All activities in order:
  [0] id=1, DAWN, pos=(442017, 388133), window=[60, 276]
  [1] id=2, Group 8, pos=(455942, 387053), window=[108, 216]
  [2] id=3, Group 4, pos=(444026, 382607), window=[84, 276]
  [3] id=4, Group 4, pos=(457341, 380889), window=[84, 276]
  [4] id=5, Group 1, pos=(452779, 386207), window=[0, 288]
  [5] id=0, DUSK, pos=(452779, 386207), window=[0, 288]


/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/3706513209.py:21: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  activities_array[array_idx].des_duration = int(individual[f"dur_g{group}"]) if pd.notna(individual[f"dur_g{group}"]).item() else 0
/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_5810/3706513209.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  activities_array[array_idx].des_start_time = int(individual[f"start_g{group}"]) if not pd.isna(individual[f"start_g{group}"]).item() else 0


In [181]:
# Run the DP algorithm with the FIXED activities
print("="*70)
print("RUNNING DP WITH FIXED ACTIVITIES")
print("="*70)

best_label_fixed = run_dp(lib, activities_array_fixed, max_num_activities_fixed, params)

if best_label_fixed:
    print("\n✓ SUCCESS! Found a feasible solution!")
    print(f"Final utility: {best_label_fixed.contents.utility:.2f}")
    
    # Extract and display the schedule
    schedule_df = extract_schedule(best_label_fixed, activities_array_fixed)
    print("\nSchedule:")
    print(schedule_df.to_string())
else:
    print("\n✗ Still no feasible solution. Need to investigate further...")
    print("Possible remaining issues:")
    print("  - Battery constraints too tight")
    print("  - Time windows incompatible")
    print("  - Activities too far apart")

RUNNING DP WITH FIXED ACTIVITIES

Running DP Algorithm...
Number of activities: 6
Horizon: 288 intervals (24.0 hours)
Starting DP...

 Solution is not feasible, check activities parameters.
 Solution is not feasible, check activities parameters.C main() returned: 0

DP completed in 0.00 seconds
ERROR: No feasible solution found!

✗ Still no feasible solution. Need to investigate further...
Possible remaining issues:
  - Battery constraints too tight
  - Time windows incompatible
  - Activities too far apart
